In [9]:
import numpy as np
import h5py as h5
import sys
import physical_data as const
from read_parameter_file import read_params

In [10]:
parameters = read_params('dummy.par')

In [11]:
ions_to_do = which_ions_to_do = [parameters.doH1,parameters.doHe2,parameters.doC2,
parameters.doC3,parameters.doC4,parameters.doN2,parameters.doN3,parameters.doN4,
parameters.doN5,parameters.doO1,parameters.doO3,parameters.doO4,parameters.doO5,
parameters.doO6,parameters.doO7,parameters.doMg2,parameters.doNe8,parameters.doAl2,
parameters.doAl3,parameters.doSi2,parameters.doSi3,parameters.doSi4,parameters.doS5,
parameters.doFe2,parameters.doFe3,parameters.do21cm]

In [12]:
for i in range(len(ions_to_do)):
    ions_to_do[i] = bool(ions_to_do[i]==1)

In [13]:
ions_to_do

[True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False]

In [14]:
ionbal_names = np.array(["h1","he2","c2","c3","c4","n2","n3","n4","n5","o1","o3","o4","o5","o6",  #This elements of this list must be in the same order as
                "o7","mg2","ne8","al2","al3","si2","si3","si4","s5","fe2","fe3","21cm"]) # ions_to_do


In [15]:
ionbal_names

array(['h1', 'he2', 'c2', 'c3', 'c4', 'n2', 'n3', 'n4', 'n5', 'o1', 'o3',
       'o4', 'o5', 'o6', 'o7', 'mg2', 'ne8', 'al2', 'al3', 'si2', 'si3',
       'si4', 's5', 'fe2', 'fe3', '21cm'], dtype='|S4')

In [16]:
ionbal_to_use = ionbal_names[ions_to_do]

In [24]:
ionss = []
first_iteration = True
for ion_file in ionbal_to_use:
    with h5.File(parameters.ibdir+ion_file+".hdf5","r") as f:
        if first_iteration:
            first_iteration = False
            z_ranges_table = f["/redshift"][...]
            logt_table = f["/logt"][...]
            logd_talbe = f["/logd"][...]
        print(ion_file)
        ionss.append([ion_file,f["/ionbal"][...]])

h1
n2
n5
ne8
si2
s5


In [23]:
parameters.ibdir

'/cosma5/data/dp004/dc-wije1/bertone_tables/ionbal/'

In [26]:
z_ranges_table

array([0.    , 0.0491, 0.1006, 0.1547, 0.2114, 0.2709, 0.3333, 0.3988,
       0.4675, 0.5396, 0.6152, 0.6945, 0.7778, 0.8651, 0.9567, 1.053 ,
       1.154 , 1.259 , 1.37  , 1.487 , 1.609 , 1.737 , 1.871 , 2.013 ,
       2.16  , 2.316 , 2.479 , 2.649 , 2.829 , 3.017 , 3.214 , 3.421 ,
       3.638 , 3.866 , 4.105 , 4.356 , 4.619 , 4.895 , 5.184 , 5.488 ,
       5.807 , 6.141 , 6.492 , 6.859 , 7.246 , 7.65  , 8.075 , 8.521 ,
       8.989 ], dtype=float32)